<a href="https://colab.research.google.com/github/kpp-bigdata/128-356-Big-Data/blob/main/lab_class_4_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ชื่อสมาชิกกลุ่ม

วรันธร อินจีน 6704800008

ธนภัทร สมพงษ์ 6704800009

รุจิภาส สุทธิศักดิ๋ 6704800010

กัญญาณัฐ ทิพย์กองลาศ 6704800027

ธนัญชนก วิเชียร์ปัญญา 6704800014


In [41]:
#!/bin/bash
!curl -L -o ./brazilian-ecommerce.zip\
  https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 42.6M  100 42.6M    0     0  49.0M      0 --:--:-- --:--:-- --:--:-- 49.0M


In [ ]:
!unzip /content/brazilian-ecommerce.zip

Archive:  /content/brazilian-ecommerce.zip
replace olist_customers_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### 🛠️ Task 1: Load & Clean Data

1. อ่านไฟล์ CSV ทั้ง 4 ไฟล์เข้าสู่ Spark DataFrame
2. ตรวจสอบ Schema และจัดการ Type (ถ้าจำเป็น)
3. สร้าง Temp View เพื่อเตรียมพร้อมสำหรับ SQL

In [ ]:
from pyspark.sql import SparkSession
# 1. เริ่มต้น Spark Session
spark = SparkSession.builder.appName("OlistEcommerce").getOrCreate()
# 2. อ่านไฟล์ CSV ทั้ง 4 ไฟล์ (ใช้ inferSchema=True เพื่อให้ Spark เดาชนิดข้อมูลอัตโนมัติ)
# ไฟล์คำสั่งซื้อ (Orders)
df_orders = spark.read.csv("olist_orders_dataset.csv", header=True, inferSchema=True)
# ไฟล์รายการสินค้าในคำสั่งซื้อ (Order Items)
df_items = spark.read.csv("olist_order_items_dataset.csv", header=True, inferSchema=True)
# ไฟล์ข้อมูลลูกค้า (Customers)
df_customers = spark.read.csv("olist_customers_dataset.csv", header=True, inferSchema=True)
# ไฟล์ข้อมูลสินค้า (Products)
df_products = spark.read.csv("olist_products_dataset.csv", header=True, inferSchema=True)
# (Optional) ลองแสดงข้อมูลดู 5 แถวแรกเพื่อตรวจสอบว่าอ่านได้ถูกต้อง
print("Orders Data:")
df_orders.show(5)

In [ ]:
from pyspark.sql.functions import col, to_timestamp

# --- 1. ตรวจสอบ Schema ปัจจุบัน ---
print("=== Schema ของ Orders (ก่อนแก้) ===")
df_orders.printSchema()
# ลองสังเกตคอลัมน์พวก timestamp ดูครับว่าเป็น 'string' หรือ 'timestamp'

# --- 2. จัดการ Type (แปลง String เป็น Timestamp) ---
# รายชื่อคอลัมน์ในตาราง Orders ที่เกี่ยวกับเวลา
date_columns = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

# วนลูปเพื่อแปลงทุกคอลัมน์ในลิสต์ให้เป็น Timestamp ที่ถูกต้อง
for column in date_columns:
    # ใช้ to_timestamp เพื่อแปลง format ให้ Spark เข้าใจว่าเป็นเวลา
    df_orders = df_orders.withColumn(column, to_timestamp(col(column)))

# --- 3. ตรวจสอบผลลัพธ์ ---
print("\n=== Schema ของ Orders (หลังแก้) ===")
df_orders.printSchema()
# ตอนนี้คอลัมน์เหล่านั้นควรจะเป็น 'timestamp' แล้วครับ

In [ ]:
# --- 3. สร้าง Temp View เพื่อเตรียมพร้อมสำหรับ SQL ---

# การตั้งชื่อ View (ในวงเล็บ) ควรตั้งให้สั้นและสื่อความหมาย
# เพราะเราต้องใช้ชื่อนี้เวลาเขียน SQL ครับ (เช่น FROM orders)

df_orders.createOrReplaceTempView("orders")
df_items.createOrReplaceTempView("order_items")
df_customers.createOrReplaceTempView("customers")
df_products.createOrReplaceTempView("products")

print("✅ สร้าง Temp View เรียบร้อยแล้ว! ตอนนี้คุณสามารถเขียน SQL ได้เลย")

# --- ทดสอบการใช้งาน SQL ---
# ลองดึงข้อมูล 5 แถวแรกจากตาราง orders ดูครับ
print("\n--- ตัวอย่างผลลัพธ์จาก SQL ---")
spark.sql("SELECT order_id, customer_id, order_status FROM orders LIMIT 5").show()

### 🔗 Task 2: Join Data

จงสร้าง `df_master` โดยการ Join ตารางดังนี้:
1. `orders` JOIN `items` (ON order_id)
2. JOIN `products` (ON product_id)
3. JOIN `customers` (ON customer_id)

> **Tip:** ตรวจสอบจำนวนแถวหลัง Join ว่าเพิ่มขึ้นหรือลดลงผิดปกติหรือไม่


In [ ]:

df_master = df_orders.join(df_items, on="order_id", how="inner")
df_master.show(5)

In [ ]:
df_master = df_master.join(df_products, on="product_id", how="inner")
df_master.show(5)

In [40]:
df_master = df_master.join(df_customers, on="customer_id", how="inner")
df_master.show(5)

+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+-------------------+-----+-------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+--------------------+------------------------+---------------+--------------+--------------------+------------------------+---------------+--------------+
|         customer_id|          product_id|            order_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|           seller_id|shipping_limit_date|price|freight_value|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_le

### 📊 Task 3: Analytics

ตอบคำถามต่อไปนี้ (เลือกใช้ SQL หรือ PySpark ก็ได้):

1. **Top Products:** สินค้าหมวดไหน (`product_category_name`) มียอดขายรวม (price) สูงที่สุด 5 อันดับแรก?
2. **Top Cities:** เมืองไหน (`customer_city`) มีจำนวนคำสั่งซื้อมากที่สุด 5 อันดับแรก?
3. **(Optional) Monthly Sales:** ยอดขายรวมแต่ละเดือนเป็นอย่างไร? (แนวโน้ม)


In [ ]:
from pyspark.sql.functions import desc, sum, count, col

df_master = df_orders.join(df_items, on="order_id", how="inner")
df_master = df_master.join(df_products, on="product_id", how="inner")
df_master = df_master.join(df_customers, on="customer_id", how="inner")
df_master.printSchema()

print(' สินค้าหมวดมียอดขายรวม (price) สูงที่สุด 5 อันดับแรก')
top_products = df_master.groupBy("product_category_name") \
    .agg(sum("price").alias("total_sales")) \
    .orderBy(desc("total_sales")) \
    .limit(5)
top_products.show(truncate=False)

print(' เมืองไหนมีจำนวนคำสั่งซื้อมากที่สุด 5 อันดับแรก')
top_cities = df_master.groupBy("customer_city") \
    .agg(count("order_id").alias("order_count")) \
    .orderBy(desc("order_count")) \
    .limit(5)
top_cities.show(truncate=False)